In [1]:
import gmaps
import pandas as pd
import numpy as np
import json
import seaborn
import requests
import matplotlib.pyplot as plt

# Google developer API key
from api_key import gmap_api

# Access maps with unique API key
gmaps.configure(api_key=gmap_api)

In [2]:
#read in combined csvfile
all_agencies_df = pd.read_csv("../data/cleandata/all_year_agencies.csv")

#limit  data to year of natural disaster
ike_agencies_pd = all_agencies_df.loc[all_agencies_df["report_year"] == 2008, :]
ike_agencies_pd.head()

,Source.Name,report_year,agency_id,ori,agency_name,state_abbr,primary_county_id,primary_county,population,nibrs_months_reported,icpsr_zip,icpsr_lat,icpsr_lng,coordinates
4308,TX2008cde_agencies.csv,2008,19425,TX1350000,King County Sheriff's Office,TX,2749,King,257,0,79236.0,33.614159,-100.250548,"(33.614159, -100.250548)"
4309,TX2008cde_agencies.csv,2008,19561,TX1730000,Motley County Sheriff's Office,TX,2787,Motley,1142,0,79244.0,34.057899,-100.793152,"(34.057899, -100.793152)"
4310,TX2008cde_agencies.csv,2008,19209,TX0920500,Longview Police Department,TX,2706,Gregg,81842,12,75606.0,32.486397,-94.816276,"(32.486397, -94.816276)"
4311,TX2008cde_agencies.csv,2008,19639,TX1990300,Heath Police Department,TX,2813,Rockwall,8176,12,75032.0,32.898054,-96.404396,"(32.898054, -96.404396)"
4312,TX2008cde_agencies.csv,2008,19009,TX0570100,Addison Police Department,TX,2671,Dallas,16125,0,75001.0,32.766987,-96.778424,"(32.766987, -96.778424)"


In [3]:
#limit  data to state of natural disaster
tx_agencies_pd = ike_agencies_pd.loc[ike_agencies_pd["state_abbr"] == "TX", :]
tx_agencies_pd.head()

,Source.Name,report_year,agency_id,ori,agency_name,state_abbr,primary_county_id,primary_county,population,nibrs_months_reported,icpsr_zip,icpsr_lat,icpsr_lng,coordinates
4308,TX2008cde_agencies.csv,2008,19425,TX1350000,King County Sheriff's Office,TX,2749,King,257,0,79236.0,33.614159,-100.250548,"(33.614159, -100.250548)"
4309,TX2008cde_agencies.csv,2008,19561,TX1730000,Motley County Sheriff's Office,TX,2787,Motley,1142,0,79244.0,34.057899,-100.793152,"(34.057899, -100.793152)"
4310,TX2008cde_agencies.csv,2008,19209,TX0920500,Longview Police Department,TX,2706,Gregg,81842,12,75606.0,32.486397,-94.816276,"(32.486397, -94.816276)"
4311,TX2008cde_agencies.csv,2008,19639,TX1990300,Heath Police Department,TX,2813,Rockwall,8176,12,75032.0,32.898054,-96.404396,"(32.898054, -96.404396)"
4312,TX2008cde_agencies.csv,2008,19009,TX0570100,Addison Police Department,TX,2671,Dallas,16125,0,75001.0,32.766987,-96.778424,"(32.766987, -96.778424)"


In [4]:
#drop null values
clean_tx_agencies_df = tx_agencies_pd.dropna(how="any")
clean_tx_agencies_df.count()

Source.Name              1092
report_year              1092
agency_id                1092
ori                      1092
agency_name              1092
state_abbr               1092
primary_county_id        1092
primary_county           1092
population               1092
nibrs_months_reported    1092
icpsr_zip                1092
icpsr_lat                1092
icpsr_lng                1092
coordinates              1092
dtype: int64

In [5]:
# show only participating agencies
participant_tx_agencies_df = tx_agencies_pd.loc[tx_agencies_pd["nibrs_months_reported"] == 12, :]
participant_tx_agencies_df = pd.DataFrame(participant_tx_agencies_df)
participant_tx_agencies_df.head()

,Source.Name,report_year,agency_id,ori,agency_name,state_abbr,primary_county_id,primary_county,population,nibrs_months_reported,icpsr_zip,icpsr_lat,icpsr_lng,coordinates
4310,TX2008cde_agencies.csv,2008,19209,TX0920500,Longview Police Department,TX,2706,Gregg,81842,12,75606.0,32.486397,-94.816276,"(32.486397, -94.816276)"
4311,TX2008cde_agencies.csv,2008,19639,TX1990300,Heath Police Department,TX,2813,Rockwall,8176,12,75032.0,32.898054,-96.404396,"(32.898054, -96.404396)"
4339,TX2008cde_agencies.csv,2008,19541,TX1700100,Conroe Police Department,TX,2784,Montgomery,68197,12,77301.0,30.299402,-95.504008,"(30.299402, -95.504008)"
4370,TX2008cde_agencies.csv,2008,18963,TX0430600,Plano Police Department,TX,2657,Collin,282968,12,75086.0,33.194745,-96.579846,"(33.194745, -96.579846)"
4381,TX2008cde_agencies.csv,2008,19472,TX1520000,Lubbock County Sheriff's Office,TX,2766,Lubbock,33099,12,79401.0,33.611469,-101.819944,"(33.611469, -101.819944)"


In [6]:
# Store 'Lat' and 'Lng' into  locations 
part_tx_locations = participant_tx_agencies_df[["icpsr_lat", "icpsr_lng"]].astype(float)
part_tx_locations.head()

,icpsr_lat,icpsr_lng
4310,32.486397,-94.816276
4311,32.898054,-96.404396
4339,30.299402,-95.504008
4370,33.194745,-96.579846
4381,33.611469,-101.819944


In [7]:
# Store 'Lat' and 'Lng' into  locations 
tx_locations = clean_tx_agencies_df[["icpsr_lat", "icpsr_lng"]].astype(float)
tx_locations.head()

,icpsr_lat,icpsr_lng
4308,33.614159,-100.250548
4309,34.057899,-100.793152
4310,32.486397,-94.816276
4311,32.898054,-96.404396
4312,32.766987,-96.778424


In [8]:
# Customize the size of the figure
part_figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
part_fig = gmaps.figure(layout=part_figure_layout)

In [9]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(tx_locations,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

# Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer.dissipating = False
heat_layer.max_intensity = 100
heat_layer.point_radius = 2

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(part_tx_locations)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))